In [2]:
# Impor pustaka yang diperlukan
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
import os

# --- Konfigurasi Path dan Model ---
# Sesuaikan path ini jika file model dan class_names.txt tidak berada di direktori yang sama
MODEL_FILENAME = 'models_hybrid/final_hybrid_sign_language_model_augmented.h5' # Ganti dengan nama file model Anda
CLASS_NAMES_FILENAME = 'processed_hybrid_data/class_names.txt'

# Path absolut jika file tidak di direktori yang sama dengan skrip
# Contoh:
# SCRIPT_DIR = os.path.dirname(os.path.abspath(__file__)) # Dapatkan direktori skrip saat ini
# MODEL_PATH = os.path.join(SCRIPT_DIR, 'models', MODEL_FILENAME) # Jika model ada di subfolder 'models'
# CLASS_NAMES_PATH = os.path.join(SCRIPT_DIR, 'data', CLASS_NAMES_FILENAME) # Jika class_names ada di subfolder 'data'

MODEL_PATH = MODEL_FILENAME # Asumsi file ada di direktori yang sama
CLASS_NAMES_PATH = CLASS_NAMES_FILENAME # Asumsi file ada di direktori yang sama


# Konfigurasi yang SAMA dengan saat pelatihan
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
IMAGE_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, 3)
NUM_LANDMARK_FEATURES = 42 # Sesuaikan jika Anda menggunakan x,y,z (menjadi 63)
# ------------------------------------

def load_class_names(file_path):
    """Memuat nama kelas dari file teks."""
    try:
        with open(file_path, 'r') as f:
            class_names = [line.strip() for line in f.readlines()]
        if not class_names:
            print(f"Peringatan: File nama kelas '{file_path}' kosong.")
            return None
        print(f"Nama kelas berhasil dimuat: {class_names[:5]}... (Total: {len(class_names)})")
        return class_names
    except FileNotFoundError:
        print(f"Error: File nama kelas tidak ditemukan di '{file_path}'")
        return None
    except Exception as e:
        print(f"Error saat memuat nama kelas: {e}")
        return None

def extract_landmarks_for_realtime(image_bgr, hands_solution_instance):
    """
    Mengekstrak landmark tangan dari satu gambar BGR untuk inferensi real-time.
    Fungsi ini HARUS SAMA persis dengan yang digunakan saat pelatihan.
    """
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    image_rgb.flags.writeable = False
    results = hands_solution_instance.process(image_rgb)
    image_rgb.flags.writeable = True # Kembalikan agar bisa ditulis jika ada pemrosesan lain

    if results.multi_hand_landmarks:
        hand_landmarks_data = results.multi_hand_landmarks[0] # Ambil tangan pertama
        landmarks_list = []
        for landmark in hand_landmarks_data.landmark:
            landmarks_list.append(landmark.x)
            landmarks_list.append(landmark.y)
            # Jika menggunakan Z saat pelatihan, uncomment baris berikut:
            # landmarks_list.append(landmark.z)

        # Normalisasi (relatif terhadap pergelangan tangan dan skala)
        base_x, base_y = landmarks_list[0], landmarks_list[1]
        normalized_landmarks = []
        for i in range(0, len(landmarks_list), 2): # Jika pakai Z, step jadi 3
            normalized_landmarks.append(landmarks_list[i] - base_x)
            normalized_landmarks.append(landmarks_list[i+1] - base_y)
            # Jika pakai Z:
            # if (i+2) < len(landmarks_list):
            #    normalized_landmarks.append(landmarks_list[i+2] - landmarks_list[2])


        max_val = np.max(np.abs(normalized_landmarks))
        if max_val == 0: max_val = 1e-6
        scaled_landmarks = np.array(normalized_landmarks) / max_val
        
        # Pastikan output memiliki panjang NUM_LANDMARK_FEATURES
        if len(scaled_landmarks.flatten()) == NUM_LANDMARK_FEATURES:
            return scaled_landmarks.flatten()
        else:
            # print(f"Peringatan: Jumlah landmark diekstrak ({len(scaled_landmarks.flatten())}) tidak sama dengan NUM_LANDMARK_FEATURES ({NUM_LANDMARK_FEATURES}).")
            return None
    return None

def main():
    # 1. Muat Model
    if not os.path.exists(MODEL_PATH):
        print(f"Error: File model tidak ditemukan di '{MODEL_PATH}'")
        return
    try:
        model = tf.keras.models.load_model(MODEL_PATH)
        model.summary()
        print("Model hibrida berhasil dimuat.")
    except Exception as e:
        print(f"Error saat memuat model: {e}")
        return

    # 2. Muat Nama Kelas
    class_names = load_class_names(CLASS_NAMES_PATH)
    if class_names is None or len(class_names) == 0:
        print("Tidak dapat melanjutkan tanpa nama kelas.")
        return
    num_classes_loaded = len(class_names)
    print(f"Jumlah kelas yang dimuat: {num_classes_loaded}")

    # 3. Inisialisasi MediaPipe Hands
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(
        static_image_mode=False,      # Mode video
        max_num_hands=2,
        min_detection_confidence=0.7,
        min_tracking_confidence=0.7
    )
    mp_drawing = mp.solutions.drawing_utils

    # 4. Inisialisasi Webcam
    cap = cv2.VideoCapture(0) # 0 untuk webcam internal
    if not cap.isOpened():
        print("Error: Tidak dapat membuka webcam.")
        return

    print("\nWebcam berhasil diinisialisasi. Tekan 'q' untuk keluar.")
    print("Arahkan tangan Anda ke kamera.")

    # Variabel untuk menghaluskan prediksi (opsional)
    prediction_buffer = []
    BUFFER_SIZE = 5 # Jumlah frame untuk dirata-rata

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("Tidak dapat menerima frame dari webcam.")
            break

        frame = cv2.flip(frame, 1) # Tampilan cermin
        
        # Simpan frame asli untuk ekstraksi landmark (karena resize bisa mempengaruhi akurasi landmark)
        frame_for_landmarks = frame.copy()

        # A. Pra-pemrosesan Gambar untuk input CNN
        resized_image_for_cnn = cv2.resize(frame, (IMAGE_WIDTH, IMAGE_HEIGHT))
        normalized_image_for_cnn = resized_image_for_cnn.astype('float32') / 255.0
        image_input_cnn = np.expand_dims(normalized_image_for_cnn, axis=0) # Tambah dimensi batch

        # B. Ekstraksi dan Pra-pemrosesan Landmark
        landmarks_data_realtime = extract_landmarks_for_realtime(frame_for_landmarks, hands)
        
        display_text = "Tidak ada tangan"
        confidence_text = ""

        if landmarks_data_realtime is not None:
            landmark_input_model = np.expand_dims(landmarks_data_realtime, axis=0) # Tambah dimensi batch

            # C. Lakukan Prediksi dengan Model Hibrida
            try:
                prediction_probabilities = model.predict(
                    {'image_input': image_input_cnn, 'landmark_input': landmark_input_model},
                    verbose=0 # Kurangi output log saat prediksi
                )
                predicted_class_index = np.argmax(prediction_probabilities, axis=1)[0]
                
                # # Opsional: Smoothing prediksi
                # prediction_buffer.append(predicted_class_index)
                # if len(prediction_buffer) > BUFFER_SIZE:
                #     prediction_buffer.pop(0)
                # if prediction_buffer:
                #     # Ambil kelas yang paling sering muncul dalam buffer
                #     predicted_class_index = np.bincount(prediction_buffer).argmax()

                confidence = prediction_probabilities[0][predicted_class_index] * 100

                if 0 <= predicted_class_index < num_classes_loaded:
                    predicted_class_name = class_names[predicted_class_index]
                    display_text = f"Prediksi: {predicted_class_name}"
                    confidence_text = f"Keyakinan: {confidence:.2f}%"
                else:
                    display_text = "Error: Indeks kelas tidak valid"
            except Exception as e:
                print(f"Error saat prediksi: {e}")
                display_text = "Error prediksi"

        # Gambar landmark pada frame asli (frame_for_landmarks atau frame)
        # Kita perlu memproses ulang frame dengan MediaPipe untuk mendapatkan koordinat untuk digambar
        # atau gunakan hasil `results` dari pemanggilan `hands.process` di dalam `extract_landmarks_for_realtime`
        # Untuk kesederhanaan, kita proses ulang untuk menggambar (bisa dioptimalkan)
        image_rgb_draw = cv2.cvtColor(frame_for_landmarks, cv2.COLOR_BGR2RGB)
        image_rgb_draw.flags.writeable = False
        results_draw = hands.process(image_rgb_draw)
        image_rgb_draw.flags.writeable = True
        if results_draw.multi_hand_landmarks:
            for hand_landmarks_coords in results_draw.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    frame, # Gambar di frame yang akan ditampilkan
                    hand_landmarks_coords,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                    mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2)
                )

        # Tampilkan teks prediksi pada frame
        cv2.putText(frame, display_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2, cv2.LINE_AA)
        if confidence_text:
            cv2.putText(frame, confidence_text, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)

        # Tampilkan frame
        cv2.imshow('Deteksi Bahasa Isyarat Hibrida Real-time', frame)

        if cv2.waitKey(5) & 0xFF == ord('q'):
            print("Keluar dari program...")
            break

    cap.release()
    cv2.destroyAllWindows()
    hands.close()

if __name__ == '__main__':
    main()

Model: "hybrid_sign_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_input (InputLayer)    [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)           2257984   ['image_input[0][0]']         
 tional)                                                                                          
                                                                                                  
 landmark_input (InputLayer  [(None, 42)]                 0         []                            
 )                                                                                                
                                                                                  